In [334]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import urllib.request as ureq
import csv
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [335]:
Australiaforum_url = 'https://www.australiaforum.com/visas-immigration/'
Ausforum = requests.get(Australiaforum_url)
#print(Ausforum)
soup = bs(Ausforum.text, 'html.parser')
#print(soup.prettify())
#soup.find_all('a')
#tabs = [link['href'] for link in soup.find_all('a', {'class':"n_first_a"})
#print(tabs)
#tab_titles = list(map(lambda h: h.text.strip(), tabs))
#print(tab_titles)
#for links in tabs:
    #print links.get("href")
#THIS PRINTS ALL HREF LINKS
#for link in soup.find_all('a'):
    #print(link.get('href'))

#THIS PRINTS THE TOP TABS WITH CLASS: N_FIRST_A:
#for link in soup.find_all("a", "n_first_a", href=True):
 #   print(link['href'])
    


In [336]:
title = soup.find("a", {'title': (re.compile('Last Page'))}).get('href')

In [337]:
#last page
last_num = int(re.search(r"index(\d+)\.", title).groups()[0])
print(last_num)

1861


In [338]:


thread_page_urls = []

for i in range(0, last_num):
    thread_page_urls.append('https://www.australiaforum.com/visas-immigration/index'+ str(i) + '.html')
    

print(len(thread_page_urls))

1861


In [339]:
#thread titles from all pages

titles = []
counter = 0 

# For running the full data set
# max_urls = len(thread_page_urls)
max_urls = 2
#0 and 1 are the same page
for i in range(1, max_urls):
    url = thread_page_urls[i]
    # Prints the URL every 10 iterations
    if counter % 10 == 0:
        print(str(counter) + ": " +url)
    thread_content = requests.get(url)
    thread_soup = bs(thread_content.text, 'html.parser')

    for link in thread_soup.find_all("a", {'id': (re.compile('thread_title_'))}):
        href = link.get('href')
        titles.append(href)              
    counter = counter + 1


0: https://www.australiaforum.com/visas-immigration/index1.html


In [340]:
thread_url = titles[0]
thread_page_count = get_thread_page_count(titles[0])
print(thread_url)

https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html


In [341]:
def get_thread_page_count(thread_url):
    page = requests.get(thread_url)
    soup = bs(page.content, 'html.parser')

    lastPage = 1
    pageNav = soup.select('div.pagenav')
    if len(pageNav) > 0:
        for s in pageNav[0].stripped_strings:
            m = re.search('^Page \d+ of (\d+)', s)
            if (m):
                groups = m.groups()
                lastPage = groups[0]
    return(int(lastPage))

In [373]:
def get_thread_messages(thread_url, total_posts, printing = False):
    groups = re.search(r"(.*)(\.html)", thread_url).groups()
    base_url = groups[0]
    suffix = groups[1]
    
    ids = []
    links = []
    dates = []
    usernames = []
    messages = []
    likes = []
    countries = []
    
    for i in range(1, total_posts+1):
        index = ''
        if i > 1:
            index = '-' + str(i)
        new_url = base_url + index + suffix
        if (printing == True):
            print(new_url)
        page = requests.get(new_url)
        soup = bs(page.content, 'html.parser')
        posts = soup.select('#posts table[id^="post"]')
        for i in posts:
            #print(i['id'])
            ids.append(i['id'])
            link = i.select('a[title="Link to this Post"]')
            #print(link[0]['href'])
            links.append(link[0]['href'])
            tds = i.find_all('td')
            #print(tds)
            dateStrings = tds[0].stripped_strings
            date = ''
            for j in dateStrings:
                m = re.search('^\d{2}-', j)
                if (m):
                    date = j
                    #print(j)
            dates.append(date)
            country = ''
            countryS = i.select('div.smallfont')
            for r in countryS:
                r = r.stripped_strings
                for c in r:
                    d = re.search('^From\ (.*)', c)
                    if (d):
                        country = d.group(1)
                        #print(country)
            countries.append(country)
            like = ''
            likeS = i.select('div.smallfont')
            for l in likeS:
                l = l.stripped_strings
                for u in l:
                    e = re.search('^\d\ likes received', u)
                    if (e):
                        like = u
                        print (like)
            usernameA = i.select('a.bigusername')
            #print(usernameA[0].string)
            usernames.append(usernameA[0].string)
            postId = '#post_message_' + i['id'][4:]
            postMsg = i.select(postId)
            concat_message = ''
            for s in postMsg[0].strings:
                # print(s)
                concat_message += s + ' ' 
            messages.append(concat_message)
    data = { "ID": ids, "Link": links, "Date": dates, "Username": usernames, "Message": messages, "Country": countries, "like": likes}


In [374]:
# Create empty frame
base_data_frame = pd.DataFrame({ "ID": [], "Link": [], "Date": [], "Username": [], "Message": [], "Country": [], "Like": []})

# For all the threads, get all the messages
for thread_url in titles:
    print(thread_url)
    thread_page_count = get_thread_page_count(thread_url)
     #For testing, reset thread_page_count to a low value
    thread_page_count = 3
    data_frame = get_thread_messages(thread_url, thread_page_count, printing=True)
    base_data_frame = base_data_frame.append(data_frame, ignore_index=True)
print('Finished')

https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html
https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html
3 likes received
1 likes received
1 likes received
1 likes received
1 likes received
https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification-2.html
5 likes received
1 likes received
5 likes received
1 likes received
3 likes received
1 likes received
https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification-3.html
4 likes received
https://www.australiaforum.com/visas-immigration/252474-registered-migration-agents.html
https://www.australiaforum.com/visas-immigration/252474-registered-migration-agents.html
https://www.australiaforum.com/visas-immigration/252474-registered-migration-agents-2.html
https://www.australiaforum.com/visas-immigration/25

2 likes received
7 likes received
7 likes received
7 likes received
7 likes received
https://www.australiaforum.com/visas-immigration/283617-partner-visa-progress-2.html
6 likes received
https://www.australiaforum.com/visas-immigration/283617-partner-visa-progress-3.html
1 likes received
7 likes received
1 likes received
1 likes received
https://www.australiaforum.com/visas-immigration/168682-skills-assessment-architecture-aaca.html
https://www.australiaforum.com/visas-immigration/168682-skills-assessment-architecture-aaca.html
https://www.australiaforum.com/visas-immigration/168682-skills-assessment-architecture-aaca-2.html
https://www.australiaforum.com/visas-immigration/168682-skills-assessment-architecture-aaca-3.html
https://www.australiaforum.com/visas-immigration/247586-461-visa-renewal-after-relationship-breakdown-any-success-stories.html
https://www.australiaforum.com/visas-immigration/247586-461-visa-renewal-after-relationship-breakdown-any-success-stories.html
https://www.au